## Thu thập dữ liệu gồm OHLC data và corporate action data
- Quá khứ đến 1.10.2025
- Tất cả nếu lấy sau ngày 1.10 thì tuyệt đối không đụng đến để đoạn từ 1.10 đến
31.12 sẽ bung ra sau (giả thực tế)
## Xử lý dữ liệu
- Stock Split
- Dividends
- Missing values & outliers
## Technical Indicators and Build Signal
- Stock Return (Log Returns)
- Build Momentum Signal (tín hiệu có thể là vị trí của giá so với Simple Moving Average (SMA)
hoặc Bollinger Bands)
## Cross-sectional
- Ranking
- Long/Short Position (Mua/Bán n cổ phiếu có thứ hạng cao đến thấp/thấp đến cao, Equal Weighting)
- Rebalancing (Giữ danh mục đầu tư này trong 1 tháng. Vào cuối tháng, đóng tất cả các vị thế, tính toán lại signal,
ranking lại, và mở các vị thế long/short mới cho tháng tiếp theo.)
## Backtesting
- Tính lợi nhuận hàng tháng của danh mục. Lợi nhuận của danh mục là trung bình của lợi nhuận từ các vị thế long và short.
- Statistical Analysis (Hypothesis Testing, "Alpha" - lợi nhuận vượt trội so với thị trường chuẩn, sử dụng QQ Plot để kiểm tra xem nó có "đuôi béo" (fat tails) hay không - cho thấy rủi ro cao hơn so với phân phối chuẩn.)
- Regression (tránh Bias)

### Lưu ý
- Lúc làm data lấy từ 1.2015 đến 31.9.2025
- Pick random 80:20/ 70:30
- 6 tháng / 3 tháng
- 6.2016 - 12.2016, 1.2018-6.2018, 3.2021-12.2021


In [ ]:
import pandas as pd
import numpy as np
import yfinance as yf
import matplotlib.pyplot as plt

In [ ]:
print("Bắt đầu thực hiện chiến lược Momentum...")

# THU THẬP VÀ XỬ LÝ DỮ LIỆU
# -------------------------------------------------
tickers = ['AAPL', 'MSFT', 'GOOG', 'AMZN', 'JPM', 'JNJ', 'XOM', 'PG', 'NVDA', 'V', 'TSLA', 'WMT']
benchmark = 'SPY'
all_tickers = tickers + [benchmark]
start_date = '2013-01-01'
end_date = '2025-10-01'

# Tải dữ liệu 'Adj Close', đã tự động xử lý chia tách cổ phiếu và cổ tức[cite: 215, 254].
try:
    data = yf.download(all_tickers, start=start_date, end=end_date)['Adj Close']
    print("Tải dữ liệu thành công.")
except Exception as e:
    print(f"Lỗi khi tải dữ liệu: {e}")
    exit()

# Xử lý missing value
data = data.ffill()

# Lấy giá cuối tháng để tính lợi nhuận hàng thán
monthly_prices = data.resample('M').last()

# GIAI ĐOẠN 2: TÍNH TOÁN LỢI NHUẬN VÀ SIGNAL
# -------------------------------------------------

# Tính lợi nhuận log hàng tháng cho tất cả các mã
# Rt = ln(Pt / Pt-1)
monthly_log_returns = np.log(monthly_prices / monthly_prices.shift(1))

# Tách riêng cổ phiếu và benchmark
returns_universe = monthly_log_returns[tickers]
returns_benchmark = monthly_log_returns[benchmark]

# Tạo tín hiệu: Signal của tháng này là lợi nhuận của tháng TRƯỚC.
# Đây là bước quan trọng để tránh "sai lệch nhìn về tương lai" (lookahead bias)
signal = returns_universe.shift(1)

# GIAI ĐOẠN 3: XÂY DỰNG DANH MỤC (RANKING STOCKS)
# -------------------------------------------------

# Xác định ngưỡng (ví dụ: 20% hàng đầu và 20% hàng cuối)
top_quintile = 0.8
bottom_quintile = 0.2

# Xếp hạng các cổ phiếu mỗi tháng dựa trên tín hiệu (lợi nhuận tháng trước)
# axis=1 thực hiện xếp hạng "đa cổ phiếu" (cross-sectionally)
ranks = signal.rank(axis=1, ascending=True, pct=True)

# Tạo vị thế: Long các cổ phiếu có hạng cao nhất (top 20%)
# (Sử dụng np.where để tạo DataFrame vị thế)
positions_long = pd.DataFrame(np.where(ranks > top_quintile, 1, 0), index=ranks.index, columns=ranks.columns)

# Tạo vị thế: Short các cổ phiếu có hạng thấp nhất (bottom 20%)
positions_short = pd.DataFrame(np.where(ranks < bottom_quintile, -1, 0), index=ranks.index, columns=ranks.columns)

# GIAI ĐOẠN 4: KIỂM THỬ LỊCH SỬ (BACKTESTING)
# -------------------------------------------------
print("Giai đoạn 4: Đang thực hiện Backtesting...")

# Tính lợi nhuận của danh mục Long (giả định trọng số bằng nhau)
# Chúng ta nhân vị thế (0 hoặc 1) với lợi nhuận THỰC TẾ (không phải signal)
# .mean(axis=1) tính trung bình lợi nhuận của các cổ phiếu được chọn mỗi tháng
long_returns = (positions_long * returns_universe).mean(axis=1)

# Tính lợi nhuận của danh mục Short (giả định trọng số bằng nhau)
# Lợi nhuận từ việc short = -1 * lợi nhuận cổ phiếu
# (positions_short là -1 hoặc 0)
short_returns = (positions_short * returns_universe).mean(axis=1)

# Lợi nhuận chiến lược là tổng của cả hai bên (long và short)
# (Lưu ý: short_returns đã là -1 * return, vì vậy chúng ta cộng chúng lại)
strategy_log_returns = long_returns + short_returns

# Tính lợi nhuận cộng dồn (vì dùng log, chúng ta cộng lại, sau đó dùng exp)
cumulative_strategy_returns = np.exp(strategy_log_returns.cumsum())
cumulative_benchmark_returns = np.exp(returns_benchmark.cumsum())

# XỬ LÝ KẾT QUẢ VÀ VẼ BIỂU ĐỒ
# -------------------------------------------------

plt.figure(figsize=(12, 7))
cumulative_strategy_returns.plot(label='Chiến lược Momentum (Long/Short)')
cumulative_benchmark_returns.plot(label=f'Benchmark ({benchmark})', linestyle='--')
plt.title('Hiệu suất Chiến lược Momentum Đa Cổ phiếu (2013-2023)')
plt.ylabel('Lợi nhuận Cộng dồn (Log scale)')
plt.xlabel('Ngày')
plt.yscale('log')  # Sử dụng thang log để dễ so sánh
plt.legend()
plt.tight_layout()
plt.grid(True, which="both", ls="--", linewidth=0.5)

# Lưu biểu đồ
output_image_path = "momentum_strategy_backtest.png"
plt.savefig(output_image_path)

print(f"Biểu đồ đã được lưu vào: {output_image_path}")